In [1]:
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm as tqdm
from keras.models import load_model
from keras import backend as K
from datetime import datetime

from Dental_Tool.Data_processing import *
from Dental_Tool.Dental_Model import *
from Dental_Tool.Process_results import *
from Dental_Tool.Dataloader import *
from Dental_Tool.KFold_v2 import *

import keras.backend as K 
import matplotlib.pyplot as plt
import numpy as np
import keras
import time

Using TensorFlow backend.


In [3]:
directory = [ 
                "Dental_Data/PBL/10_20200901", "Dental_Data/PBL/10_20200901_Flip", 
                "Dental_Data/PBL/10_clahe_20200901", "Dental_Data/PBL/10_clahe_20200901_Flip"
            ]


directory = [ i + "/mapping.json" for i in directory]
argscale_num = 80

data = load_json(directory)
dataset = json_2_dataframe_PBL(data)
dataset

In [149]:
def capnet(input_shape=(28,28,1))
        caps1_n_maps = 32                      # 32 Feature map
        caps1_n_caps = caps1_n_maps * (6 * 6)  # 1152 primary capsule, each FeatureMap contain 
                                               # 6 * 6 capsules  
        caps1_n_dims = 8                       # each capsule output 8 dim vector
        
        conv1_params = {
            "filters": 256,
            "kernel_size": 9,
            "strides": 1,
            "padding": "valid",
            "activation": "relu",
        }
        
        conv2_params = {
            "filters": caps1_n_maps * caps1_n_dims, # 256 个卷积滤波器
            "kernel_size": 9,
            "strides": 2,
            "padding": "valid",
            "activation": "relu"
        }
        
        input_shape = Input(shape=input_shape)  # size of input image is 28*28

        # a convolution layer output shape = 20*20*256
        conv1 = Conv2D(name="conv1", **conv1_params)(input_shape)

        # convolution layer with stride 2 and 256 filters of size 9*9
        conv2 = Conv2D(name="conv2", **conv2_params)(conv1)

        # reshape into 1152 capsules of 8 dimensional vectors
        reshaped = Reshape((6*6*32,8))(conv2)

        # squash the reshaped output to make length of vector b/w 0 and 1
        squashed_output = Lambda(squash)(reshaped)


def squash(inputs):
        # take norm of input vectors
        squared_norm = K.sum(K.square(inputs), axis = -1, keepdims = True)

        # use the formula for non-linear function to return squashed output
        return ((squared_norm/(1+squared_norm))/(K.sqrt(squared_norm+K.epsilon())))*inputs

    

 

class DigitCapsuleLayer(Layer):
        # creating a layer class in keras
        def __init__(self, **kwargs):
            super(DigitCapsuleLayer, self).__init__(**kwargs)
            self.kernel_initializer = initializers.get('glorot_uniform')

        def build(self, input_shape): 
            # initialize weight matrix for each capsule in lower layer
            self.W = self.add_weight(shape = [10, 6*6*32, 16, 8], initializer = self.kernel_initializer, name = 'weights')
            self.built = True

        def call(self, inputs):
            inputs = K.expand_dims(inputs, 1)
            inputs = K.tile(inputs, [1, 10, 1, 1])
            # matrix multiplication b/w previous layer output and weight matrix
            inputs = K.map_fn(lambda x: K.batch_dot(x, self.W, [2, 3]), elems=inputs)
            b = tf.zeros(shape = [K.shape(inputs)[0], 10, 6*6*32])

    # routing algorithm with updating coupling coefficient c, using scalar product b/w input capsule and output capsule
            for i in range(3-1):
                c = tf.nn.softmax(b, dim=1)
                s = K.batch_dot(c, inputs, [2, 2])
                v = squash(s)
                b = b + K.batch_dot(v, inputs, [2,3])

            return v 
        def compute_output_shape(self, input_shape):
            return tuple([None, 10, 16])

        def output_layer(inputs):
            return K.sqrt(K.sum(K.square(inputs), -1) + K.epsilon())

def loss_fn(y_true, y_pred):
 
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))
 
    return K.mean(K.sum(L, 1))

digit_caps = DigitCapsuleLayer()(squashed_output)
outputs = Lambda(output_layer)(digit_caps)




(None, None, 8)
(None, None, 1, 8)
(None, None, 3, 8)
(1152, 3, 1, 16)
(1152, 3, 8, 16)


ValueError: Inconsistent shapes: saw (1152, 8, 8, 16) but expected (1152, 3, 1, 16) 